In [1]:
%%javascript
    $("#homeButton").remove()
    $('body').append('<a href="#'+$("h1,h2,h3:eq(0)").attr("id")+'" style="position: fixed; bottom: 10px; right: 10px;" type="button" class="btn btn-success btn-circle btn-lg"><i class="glyphicon glyphicon-link" id="homeButton"></i></a>');
    $("#MainMenu").remove()
    var menu = 
      '<div id="MainMenu" style="position: fixed; top: 120px; right: 10px; z-index:6; max-height: 500px;">'+
        '<div class="list-group panel" >'+
          '<a href="#collapseMenu" class="list-group-item list-group-item-success" data-toggle="collapse" data-parent="#MainMenu">Indíce<i class="fa fa-caret-down"></i></a>'+
          '<div class="collapse" id="collapseMenu" style="overflow-y: overlay; max-height: 500px;">'+
          '</div>'+
        '</div>'+
      '</div>'
   
    var parent = $(menu)
    var arrayIds = []
    $("h1,h2,h3").attr("id",function(id,Value){
        if(Value != ""){
            var content = (Value.replace(new RegExp('-', 'g'), ' '));
            content = "&nbsp;".repeat(parseInt($(this).prop("tagName")[1])*2-1) + content;
            $(parent).find("#collapseMenu").append('<a href="#'+Value+'" style="position:relative;" class="list-group-item" data-parent="#SubMenu1">'+content+'</a>');
        }
    })
$('body').append(parent)

<IPython.core.display.Javascript object>

In [2]:
%%html
<style>
    h3{
        color:#0094E4;
    }
    h2{
         color:#F28773;
    }
    h1{
         color:#0094E4;
    }
</style>

<img src="./images/logo.png" style="width: 20%; margin-left: 70%;">
<img src="./images/python.png">

<h1>Hitchhiker's Guide to Data Science: Random Forests</h1>

## Bibliotecas generales

In [3]:
from pyspark.sql.functions import col as c

# Random Forest

El Random Forest (o bosque aleatorio) está constituido por árboles de decisión.

Entonces antes de empezar:

# Árboles de decisión

Un árbol de decisión es una forma gráfica y analítica
de representar todos los eventos (sucesos) que
pueden surgir a partir de una decisión asumida en
cierto momento.

Los árboles de decisión ayudan a tomar la decisión “más acertada”,
desde un punto de vista probabilístico, ante un
abanico de posibles decisiones. Permiten desplegar visualmente un problema y organizar los cálculos que deben realizarse.

A partir de un set de datos se puede inferir un árbol de decisión ya sea regresor o clasificador. La diferencia escencial es que un árbol de regresión se usa para predecir respuestas cuantitativas y uno de clasificación respuestas cualitativas.

| Clasificación                                                                                                                                                                                    | Regresión                                                                                                                     |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| Predecir respuestas cualitativas                                                                                                                                                                 | Predecir respuestas cuantitativas                                                                                             |
| Predecimos que cada observación pertenece a la clase que ocurre más comunmente en la región a la que pertenece.                                                                                  | La respuesta predicha está dada por la respuesta promedio de las observaciones de entrenamiento que pertenecen al mismo nodo. |
| Por lo general nos interesa no  sólo la clase (target) que pertenece a un nodo terminal, sino también las proporciones de clases de las  observaciones de entreenamiento que  caen en esa región |                                                                                                                               |

Para fines de fines de este mini curso, nos concentraremos en el modo clasificador.


## Ventajas
* Pueden usarse para clasificación y regresión
* Son fáciles de interpretar
* Pueden manejar datos categóricos
* Por lo general no requieren escalar los datos
* Pueden capturar no lineariades e interacciones entre features

## Componentes

<div class="alert alert-warning"><strong>Nodo de decisión</strong><br>
Indica que una decisón debe tomarse en el proceso, en general (no siempre) se representa con un cuadro
</div>

<div class="alert alert-warning"><strong>Nodo de probabilidad</strong><br>
Indica que en ese punto del proceso ocurre un evento aleatorio. Suele representarse con un círculo
</div>

<div class="alert alert-warning"><strong>Rama</strong><br>
Nos muestra los distintos caminos que se pueden emprender cuando se toma una decisión u ocurre un evento aleatorio, en general se representa con flechas
</div>

<img src="./images/arbol_d_decision.png">

## Pasos para construir un Árbol de Decisión
* Definir el problema.
* Dibujar el árbol de decisión.
* Asignar probabilidades a los eventos aleatorios.
* Estimar los resultados para cada combinación posible de alternativas.
* Resolver el problema obteniendo como solución la ruta más óptima.

### Ejemplo:
<div class="alert alert-success alert-dismissable" id="def2">
<a href="#" class="close" data-dismiss="alert" aria-label="close">&times;</a><strong>
Suponiendo que nuestro problema es decidir si debemos conceder o no un préstamo...</strong>
</div>

<img src="./images/arbol_prestamo.png">


## Un ejemplo en sklearn y python

**NOTA, NO ES EL OBJETIVO DE ESTE CURSO APRENDER SKLEARN, PERO ES VISULAMENTE ÚTIL PARA ENTENDER LO QUE QUEREMOS HACER**

**Problema:** Queremos predecir la especie de una flor iris como setosa, versicolor o virginica con datos de la longitud de sus pétalos y sépalos. 

<img src="./images/petal_sepal.png">

<img src="./images/iris-flowers.png">

1. Cargamos el iris data set y el clasificador por árboles de decisión

In [8]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

In [9]:
iris= load_iris()

In [10]:
type(iris)

sklearn.datasets.base.Bunch

In [11]:
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
#data1
#iris.data

### Llamamos el clasificador de árbol de decisión

In [12]:
tree_clf = DecisionTreeClassifier(max_depth=2)

In [13]:
x = iris.data[: ,2:]
y = iris.target

tree_clf.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

### Hacemos la imagen

In [14]:
from sklearn.tree import export_graphviz

In [16]:
export_graphviz(
    tree_clf,
    out_file = "./images/iris_tree.dot",
    feature_names = iris.feature_names[2:],
    class_names = iris.target_names,
    rounded = True,
    filled = True
)

<img src="./images/iris_tree.png">

* **samples:** ¿Para cuántas instancias del conjunto de entrenamiento aplica esta regla?
Por ejemplo 50 instancias tienen un ancho de pétalo menor o igual a 0.8.
* **value:** Para cuántas instancias del conjunto de entrenamiento de cada clase aplica este nodo.
* **gini:** (No confundir con Gini score) Es una medida de la impureza del nodo. Un nodo es puro (gini=0), si todas las instancias del entrenamiento para las que aplica son de la misma clase.

Impureza de Gini para el iésimo nodo:

$G_i = 1- \sum_{k=1}^{n}p_{i,k}^2$

donde:

$p_{i,k}$ Es el cociente de los "values" de cada clase entre el "samples" del nodo i.

Ejemplo:

En el nodo de la izquierda del segundo nivel

$G_i = 1- ((\frac{0}{54})^2 + (\frac{49}{54})^2 + (\frac{5}{54})^2)$

**Nota:** Gini no es la única medida de pureza.

### Para hacer predicciones con el modelo:

1.Nos situamos en el nodo base (profundidad = 0) hasta arriba. Ahí preguntamos si los pétalos tienen cierta característica (en este caso una ancho de pétalo menor o igual que 0.8 cm.

<img src="./images/iris_tree_setosa.png">

2.Si, en este caso resulta ser que el ancho del pétalo es menor a 0.8cm, voy a la izquierda y ya terminé: mi planta iris es de la clase setosa.

3.Si mi planta no cumple con tener un pétalo de ancho menor a 0.8cm, de nuevo, tengo que moverme al primer nivel de profundidad.

<img src="./images/iris_tree_versicolor_virginica.png">
El nodo al que llego no es un nodo hoja, sino de decisión, así que dada la condicional , reviso el ancho del pétalo, ¿es menor o igual a 1.75?

4.Si el ancho del pétalo fue menor a 1.75 cm voy a la izquierda, clasifico mi planta como versicolor y terminé.
<img src="./images/iris_tree_versicolor.png">

5.Si por el contrario, el ancho de mi pétalo fue mayor a 1.75cm
<img src="./images/iris_tree_virginica.png">

### Para hacer la predicción *in silico*

Vamos a suponer que encontramos una flor con pétalos de 5cm de largo y 1.5cm de ancho, bajo el modelo que estamos viendo debería de corresponder al nodo de profundidad 2 de la izquierda y darnos las siguientes probabilidades: 0 para iris setosa (0/54), .907 para iris versicolor (49/54) y alrededor de .93 para iris virginica (5/54).

<div class="alert alert-danger" role="alert">
  Esto quiere decir, ¿cuál es la probabilidad de que la flor que encontre sea de un tipo determinado dado mi árbol de decisión?
</div>

In [20]:
tree_clf.predict_proba([[5,.15]])

array([[ 1.,  0.,  0.]])

**¿Por qué sale distinto a nuestra predicción?**

<div class="alert alert-danger" role="alert">
  Los árboles de decisión tienen muchas ventajas: son fáciles de usar, fáciles de entender e interpretar, versátiles y poderosos, pero <strong>son muy sensibles al conjunto de entrenamiento</strong> y, al menos scikit learn es estocástico puedes obtenr modelos muy diferentes con el mismo set de datos. A menos que fijes el parámetros random_state.
</div>

Dado el que sea tan lábil, podemos aprovechar esta propiedad para hacer muchos árboles de decisión y buscar el mejor posible, ese es el principio detrás de RandomForest.

### Chequemos de nuevo sus ventajas y desventajas:

#### Ventajas:
* Fáciles de explicar
* Se ha llegado a decir que son el método que más se parece a las decisiones tomadas por humanos.
* Pueden desplegarse gráficamente y explicarse a personas no expertas.
* Pueden manejar sin problemas variables cualitativas sin tener que crear variables *dummy*

#### Desventajas:
* Un árbol por si solo, en genral no tiene el mismo poder predictivo que otros métodos de regresión y clasificación.
* Pueden ser muy poco robustos (un pequeño cambio en los datos puede significar un gran cambio en el árbol final).

## Primero hay que correr Spark context y Spark session

In [17]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
spark =SparkSession.builder.appName('lreg').getOrCreate()

### Objetivo
Predecir customer churn

## Leer datos

In [3]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



## Describir datos

In [5]:
data.describe().toPandas()

,summary,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Location,Company,Churn
0,count,900,900,900,900,900,900,900,900,900
1,mean,None,41.81666666666667,10062.82403333334,0.4811111111111111,5.27315555555555,8.587777777777777,None,None,0.16666666666666666
2,stddev,None,6.127560416916251,2408.644531858096,0.4999208935073339,1.274449013194616,1.7648355920350969,None,None,0.3728852122772358
3,min,Aaron King,22.0,100.0,0,1.0,3.0,"00103 Jeffrey Crest Apt. 205 Padillaville, IA ...",Abbott-Thompson,0
4,max,Zachary Walsh,65.0,18026.01,1,9.15,14.0,Unit 9800 Box 2878 DPO AA 75157,"Zuniga, Clark and Shaffer",1


## Seleccionar datos

In [6]:
from pyspark.ml.feature import VectorAssembler

In [7]:
#assembler = VectorAssembler(inputCols = ['Age','Total_Purchase','Years','Num_Sites','Account_Manager'],outputCol='features')

In [8]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase','Years','Num_Sites'],outputCol='features')

In [9]:
assembled = assembler.transform(data)
data_VA = assembled.select('features','churn')

## Obtener conjuntos de datos de entrenamiento y de prueba

In [3]:
train_data,test_data = data_VA.randomSplit([0.7,0.3])

NameError: name 'data_VA' is not defined

## Random forest

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator


In [18]:
rf = RandomForestRegressor(labelCol='churn')

In [19]:
fit_rfModel = rf.fit(train_data)

In [20]:
training_sum = fit_rfModel.

AttributeError: 'RandomForestClassificationModel' object has no attribute 'summary'

Seems fine, let's do evaluation metrics :D

In [15]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [16]:
prediction_and_labels = fit_rfModel.evaluate(test_data)

AttributeError: 'RandomForestRegressionModel' object has no attribute 'evaluate'

In [80]:
prediction_and_labels.predictions.limit(10).toPandas()

,features,churn,rawPrediction,probability,prediction
0,"[26.0, 8787.39, 5.42, 11.0]",1,"[0.821835222748, -0.821835222748]","[0.694625768032, 0.305374231968]",0.0
1,"[27.0, 8628.8, 5.3, 7.0]",0,"[5.67155854679, -5.67155854679]","[0.996569314319, 0.00343068568105]",0.0
2,"[28.0, 11128.95, 5.12, 8.0]",0,"[4.31437686644, -4.31437686644]","[0.986801645201, 0.0131983547994]",0.0
3,"[29.0, 10203.18, 5.82, 8.0]",0,"[3.95266333937, -3.95266333937]","[0.981158337445, 0.0188416625554]",0.0
4,"[29.0, 12711.15, 5.74, 7.0]",0,"[5.00468555006, -5.00468555006]","[0.993338226987, 0.00666177301277]",0.0
5,"[29.0, 13255.05, 4.89, 8.0]",0,"[4.22000602305, -4.22000602305]","[0.985514361987, 0.0144856380134]",0.0
6,"[30.0, 6744.87, 5.14, 9.0]",0,"[3.33821829835, -3.33821829835]","[0.965716903067, 0.0342830969327]",0.0
7,"[31.0, 9574.89, 7.34, 8.0]",0,"[3.06727876925, -3.06727876925]","[0.955522665802, 0.0444773341981]",0.0
8,"[32.0, 8617.98, 6.2, 10.0]",1,"[1.30086422037, -1.30086422037]","[0.785980393986, 0.214019606014]",0.0
9,"[32.0, 11715.72, 4.73, 9.0]",0,"[3.06552303371, -3.06552303371]","[0.955447988914, 0.044552011086]",0.0


In [81]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol= 'churn')

### area under curve

In [82]:
auc = churn_eval.evaluate(prediction_and_labels.predictions)

In [83]:
auc

0.7050194414987628

### Predict on new data

In [84]:
fit_lrModel = lr.fit(data_VA)

In [85]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True,header=True)

In [86]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [87]:
test_new_customers = assembler.transform(new_customers)

In [88]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [90]:
final_results = fit_lrModel.transform(test_new_customers)

In [91]:
final_results.toPandas()

,Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,features,rawPrediction,probability,prediction
0,Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,"38612 Johnny Stravenue Nataliebury, WI 15717-8316",King Ltd,"[37.0, 9935.53, 7.71, 8.0]","[2.41832359387, -2.41832359387]","[0.918213939529, 0.0817860604711]",0.0
1,Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,"21083 Nicole Junction Suite 332, Youngport, ME...",Cannon-Benson,"[23.0, 7526.94, 9.28, 15.0]","[-5.99066587605, 5.99066587605]","[0.00249575310724, 0.997504246893]",1.0
2,Jeremy Chang,65.0,100.00,1,1.00,15.0,2006-12-11 07:48:13,"085 Austin Views Lake Julialand, WY 63726-4298",Barron-Robertson,"[65.0, 100.0, 1.0, 15.0]","[-3.43252821753, 3.43252821753]","[0.03129419877, 0.96870580123]",1.0
3,Megan Ferguson,32.0,6487.50,0,9.40,14.0,2016-10-28 05:32:13,"922 Wright Branch North Cynthialand, NC 64721",Sexton-Golden,"[32.0, 6487.5, 9.4, 14.0]","[-5.34296748644, 5.34296748644]","[0.00475890465963, 0.99524109534]",1.0
4,Taylor Young,32.0,13147.71,1,10.00,8.0,2012-03-20 00:36:46,Unit 0789 Box 0734 DPO AP 39702,Wood LLC,"[32.0, 13147.71, 10.0, 8.0]","[1.28437160096, -1.28437160096]","[0.783193000738, 0.216806999262]",0.0
5,Jessica Drake,22.0,8445.26,1,3.46,14.0,2011-02-04 19:29:27,1148 Tina Stravenue Apt. 978 South Carlos TX 2...,Parks-Robbins,"[22.0, 8445.26, 3.46, 14.0]","[-1.438520474, 1.438520474]","[0.191774566203, 0.808225433797]",1.0


## Referencias
http://www.dmae.upct.es/~mcruiz/Telem06/Teoria/arbol_decision.pdf
https://spark.apache.org/docs/2.2.0/mllib-decision-tree.html